## Importy

* **requests** - knihovna pro posílání HTTP požadavků
* **pandas** - knihovna pro práci s daty
* **xlrd** - knihovna pro práci s excel vstupem
* **IPython** - knihovna pro práci s výstupními texty
* **time** - knihovna pro práci s časem

**<font color='green'>Zde není potřeba nic měnit</font>**

In [11]:
import requests
import pandas as pd
# import xlrd
from IPython.display import clear_output
import time

## Konstanty

### Vstupní soubor s daty

* **filePath** - absolutní/relativní cesta k souboru se vstupními daty (při použití zpětných lomítek je třeba napsat \\)
* **fileType** - typ vstupního souboru s daty (je třeba změnit pouze index - 0 pro CSV, 1 pro EXCEL)
* **encoding** - v případě práce s CSV nastavte použité kódování vstupního souboru
* **delimiter** - v případě práce s CSV nastavte použitý oddělovač
* **decimal** - v případě práce s CSV zvolte oddělovač desetinných čísel (obvykle tečka nebo čárka)

### Definice adresy

* **cityColumnName** - přesné označení sloupce, který obsahuje město
* **streetColumnName** - přesné označení sloupce, který obsahuje ulici
* **streetNumberColumnName** - přesné označení sloupce, který obsahuje číslo popisné (případně oriantační či oboje)


### Výstupní soubor

* **outputFile** - absolutní/relativní cesta k souboru, do kterého budou uložena výstupní data


**<font color='red'>Zde je potřeba správně nastavit veškeré zmíněné konstanty! Nesprávná konfigurace konstant bude mít za následek chybu!</font>**

In [12]:
filePath = 'skoly_vsechny_kraje_praha_gps.csv' #ZDE ZMĚŇTE CESTU KE VSTUPNÍMU SOUBORU
fileType = ['CSV', 'EXCEL'][0] #ZDE ZMĚŇTE POUZE INDEX (0 PRO CSV, 1 PRO EXCEL)
encoding = 'utf-8' #ZDE ZMĚŇTE KÓDOVÁNÍ, ABY SEDĚLO S TÍM, KTERÉ JSTE POUŽILI VE VSTUPNÍM SOUBORU
delimiter = ';' #ZDE ZMĚŇTE ODDĚLOVAČ, ABY SEDĚL S TÍM, KTERÝ JSTE POUŽILI VE VSTUPNÍM SOUBORU
# decimal = '.' #ZDE NASTAVTE ODDĚLOVAČ DESETINNÝCH MÍST, KTERÝ JSTE POUŽILI VE VSTUPNÍM SOUBORU 

cityColumnName = 'obec' #ZDE NASTAVTE NÁZEV SLOUPCE S MĚSTEM
streetColumnName = 'nazev_ulice' #ZDE NASTAVTE NÁZEV SLOUPCE S ULICÍ
streetNumberColumnName = 'cislo_popisne' #ZDE NASTAVTE NÁZEV SLOUPCE S ČÍSLEM POPISNÝM

outputFile = 'gps.csv' #SOUBOR MUSÍ BÝT CSV

## Načtení dat

* data jsou pouze načtena a je provedena náhrada chybějících údajů

**<font color='green'>Zde není potřeba nic měnit</font>**

In [13]:
data = None
if fileType == 'EXCEL':
    data = pd.read_excel( filePath )
else:
    data = pd.read_csv( filePath, encoding = encoding, sep = delimiter )
data[cityColumnName].fillna( '', inplace = True )
data[streetColumnName].fillna( '', inplace = True )
data[streetNumberColumnName].fillna( '', inplace = True )

## Definice pomocných funkcí

* **preparePossibleAddresses** -příprava potenciálních adres 
* **getGPSLocation** - získání GPS souřadnic z adresy
* **getEstimatedTimeString** - pomocný výpis pro zbývající čas

**<font color='green'>Zde není potřeba nic měnit</font>**

In [14]:
def preparePossibleAddresses( city, street, streetNumber ):
    if 'Praha' in city:
        city = 'Praha'
    possibleStreets = list( set( [street] + street.split(' ') + street.split('.') ) )
    possibleStreets.sort(key=len, reverse = True)
    return [ str( city ) + ' ' + str( pStreet ) + ' ' + str( streetNumber ) for pStreet in possibleStreets] + [ str( city ) + ' ' + str( pStreet ) + ' ' + str( streetNumber ).split('/')[-1] for pStreet in possibleStreets]

def getGPSLocation( city, street, streetNumber ):
    preparedAddresses = preparePossibleAddresses( city, street, streetNumber )
    for address in preparedAddresses:
        try:
            URL = 'https://nominatim.openstreetmap.org/search?q=<<adresa>>&format=json'
            r = requests.get(url = URL.replace('<<adresa>>', address.replace( ' ', '+' ) ))
            data = r.json()
            return { 'latitude': data[0]['lat'], 'longitude': data[0]['lon'] }
        except Exception as e:
            pass
    return { 'latitude': 0, 'longitude': 0 }

def getEstimatedTimeString( estimatedTime ):
    hours = int( estimatedTime / 3600 )
    minutes = int( ( estimatedTime - hours * 3600 ) / 60 )
    seconds = int( estimatedTime - minutes * 60 - hours * 3600 )
    hoursStringPart = [str(hours), '0' + str(hours)][hours<10]
    minutesStringPart = [str(minutes), '0' + str(minutes)][minutes<10]
    secondsStringPart = [str(seconds), '0' + str(seconds)][seconds<10]
    return hoursStringPart + ':' + minutesStringPart + ':' + secondsStringPart

## Proces získání souřadnic

**<font color='green'>Zde není potřeba nic měnit</font>**

In [15]:
newLatitudes = []
newLongitudes = []
actualRow = 0
startTime = time.time( )
for row in data.iloc:
    GPSCoordinates = getGPSLocation( row[cityColumnName], row[streetColumnName], row[streetNumberColumnName] )
    newLatitudes.append( GPSCoordinates['latitude'] )
    newLongitudes.append( GPSCoordinates['longitude'] )
    actualTime = time.time( )
    oneRowTime = ( actualTime - startTime ) / ( actualRow + 1 )
    estimatedTime = oneRowTime * ( len( data ) - ( actualRow + 1 ) )
    clear_output(wait=True)
    print( 'Dokončeno: ', int( ( actualRow + 1 ) / len( data ) * 100 ), '% ... Odhadovaný čas: ' + getEstimatedTimeString( estimatedTime ) )
    actualRow += 1
data['Zeměpisná šířka'] = newLatitudes
data['Zeměpisná délka'] = newLongitudes
data['Zeměpisná šířka'] = data['Zeměpisná šířka'].astype( 'float64' ).round( 6 )
data['Zeměpisná délka'] = data['Zeměpisná délka'].astype( 'float64' ).round( 6 )

Dokončeno:  100 % ... Odhadovaný čas: 00:00:00


## Uložení výstupních dat

* vstupní data jsou obohacena o zeměpisnou šířku a délku
* v případě, že se pro danou adresu nepodařilo GPS souřadnice nalézt, jsou pro šířku i délku uloženy hodnoty 0
* výstupní soubor je ve formátu CSV, kódování je UTF-8 s podporou českého jazyka, oddělovač je středník a je použita desetinná tečka

**<font color='green'>Zde není potřeba nic měnit</font>**

In [16]:
data.to_csv( outputFile, encoding = 'utf-8-sig', sep = ';', index = False, decimal = '.' )

## Výpis nenalezených adres

**<font color='green'>Zde není potřeba nic měnit</font>**

In [17]:
data[(data['Zeměpisná šířka']==0)&(data['Zeměpisná délka']==0)]

,Unnamed: 0,Column1,id,nazev,typ,adresa1,adresa2,adresa3,PSC,obec,...,obor_forma,obor_delka,obor_kapacita,okres,kod_okresu_a,kod_okresu_b,nazev_ulice,cislo_popisne,Zeměpisná šířka,Zeměpisná délka


In [21]:
pd.set_option('display.max_rows', None)

In [22]:
data

,Unnamed: 0,Column1,id,nazev,typ,adresa1,adresa2,adresa3,PSC,obec,...,obor_forma,obor_delka,obor_kapacita,okres,kod_okresu_a,kod_okresu_b,nazev_ulice,cislo_popisne,Zeměpisná šířka,Zeměpisná délka
0,0,0,60076135,Střední škola,C00,Fráni Šrámka 1193/23,České Budějovice 3,370 01 České Budějovice,37001.0,České Budějovice,...,10,40,297.0,České Budějovice,CZ0311,40282,Fráni Šrámka,1193/23,48.982194,14.477236
1,1,1,60076101,Střední škola,C00,Jírovcova 1788/8,České Budějovice 3,370 01 České Budějovice,37001.0,České Budějovice,...,10,40,255.0,České Budějovice,CZ0311,40282,Jírovcova,1788/8,48.978906,14.477792
2,2,2,60076062,Střední škola,C00,Čihovice 13,Hněvkovice na levém břehu Vltavy,375 01 Týn nad Vltavou,37501.0,Týn nad Vltavou,...,10,40,30.0,České Budějovice,CZ0311,40282,Čihovice,13,49.211838,14.429222
3,3,3,62534408,Střední škola,C00,Školní 995,374 01 Trhové Sviny,374 01 Trhové Sviny,37401.0,Trhové Sviny,...,10,40,85.0,České Budějovice,CZ0311,40282,Školní,995,48.844149,14.634120
4,4,4,60076046,Střední škola,C00,Husova tř. 1849/1,České Budějovice 3,370 01 České Budějovice,37001.0,České Budějovice,...,10,40,480.0,České Budějovice,CZ0311,40282,Husova tř.,1849/1,48.977823,14.471181
5,5,6,666122,Střední škola,C00,Jirsíkova 420/5,České Budějovice 1,370 01 České Budějovice,37001.0,České Budějovice,...,10,40,180.0,České Budějovice,CZ0311,40282,Jirsíkova,420/5,48.973123,14.476519
6,6,7,45019517,Střední škola,C00,Třebízského 1010/9,České Budějovice 5,370 06 České Budějovice,37006.0,České Budějovice,...,10,40,150.0,České Budějovice,CZ0311,40282,Třebízského,1010/9,48.973080,14.504087
7,7,8,60074302,Střední škola,C00,Pražská tř. 2532/54A,České Budějovice 3,370 04 České Budějovice,37004.0,České Budějovice,...,10,40,110.0,České Budějovice,CZ0311,40282,Pražská tř.,2532/54A,48.988444,14.474059
8,8,9,60074281,Střední škola,C00,Žižkova tř. 250/4,České Budějovice 6,370 01 České Budějovice,37001.0,České Budějovice,...,10,40,100.0,České Budějovice,CZ0311,40282,Žižkova tř.,250/4,48.972343,14.479578
9,9,10,60075775,Střední škola,C00,Česká 142/64,České Budějovice 1,370 01 České Budějovice,37001.0,České Budějovice,...,10,40,132.0,České Budějovice,CZ0311,40282,Česká,142/64,48.974736,14.474285
